# Assignment 4: Evaluating Search Engines

For this assignment, we leave aside the code we developed so far, and look into the more general issue of how to evaluate and compare different search engines. The ultimate test for any Information Retrieval system is how well it is able to satisfy the information needs of users.

# Cohen's Kappa

Our evaluation will involve the calculation of [Cohen's kappa](https://en.wikipedia.org/wiki/Cohen's_kappa) to quantify the degree to which two human assessors agree or disagree on whether results are considered relevant or not. To calculate Cohen's kappa, we are going to use the [scikit-learn library](http://scikit-learn.org/stable/):

In [1]:
# pip install --user scikit-learn

In [2]:
from sklearn.metrics import cohen_kappa_score

This library expects relevance assessments as lists of elements where `1` stands for _relevant_ and `0` stands for _not relevant_, for example like this:

In [3]:
a1=[1,0,1,0,1,0,1,0]

This list means that the first document was assessed to be relevant, the second to be not relevant, the third to be relevant etc.

We need two assessments in order to calculate Cohen's kappa, so let's make another exemplary list that only differs on the last element:

In [4]:
a2=[1,0,1,0,1,0,1,1]

We can now invoke the library as follows to calculate the agreement between the two:

In [5]:
cohen_kappa_score(a1, a2)

0.75

This value represents high agreement. We can reach maximal agreement if the two assessments are identical:

In [6]:
cohen_kappa_score(a1, a1)

1.0

Now, let's see what happens for a third assessment that differs on three positions with the first one (the three last positions):

In [7]:
a3=[1,0,1,0,1,1,0,1]

cohen_kappa_score(a1, a3)

0.25

We get a smaller but still positive value, because these two assessments still mostly agree. If we make a further example that differs on 6 of the 8 positions, we get the following result:

In [8]:
a4=[1,0,0,1,0,1,0,1]

cohen_kappa_score(a1, a4)

-0.5

The score is now negative, because the two differ on more positions than they agree. The agreement is in fact less than what you would expect to occur just by chance. We get the maximal disagreement if we define a fifth example that disagrees on all positions:

In [9]:
a5=[0,1,0,1,0,1,0,1]

cohen_kappa_score(a1, a5)

-1.0

Be aware that the kappa score cannot be calculated if you have only `1`s or only `0`s:

In [10]:
a6=[1,1,1,1,1,1,1,1]
a7=[1,1,1,1,1,1,1,1]

cohen_kappa_score(a6, a7)

C:\Users\camis\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
C:\Users\camis\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:730: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)


nan

And in the case of a highly skewed set (either vast majority of agreements on `1` or vast majority of agreements on `0`), the kappa score can be counter-intuitive:

In [11]:
a8=[1,1,1,1,1,1,0,1]
a9=[1,1,1,1,1,1,1,0]

cohen_kappa_score(a8, a9)

-0.1428571428571428

Now that we understand how this function works, we will apply it below for our specific evaluation.

# Results and Assessments

Next, we will define some auxilary code to deal with lists of URLs from search engines and associated relevance assessments. We will encode result lists like this:

In [12]:
urls = [
    'https://en.wikipedia.org/wiki/Information_retrieval/',  # 1st result
    'http://www.dictionary.com/browse/information',          # 2nd result
    'https://nlp.stanford.edu/IR-book/'                      # ...
]

And we represent corresponding assessments, as above, as lists of the same size containing relevance values:

In [13]:
my_assessment = [1, 0, 1]
another_assessment = [0, 0, 1]

In order to nicely display URL lists, with or without related assessments, we define a function called `display_results`:

In [14]:
from IPython.display import display, HTML

def display_results(urls, assessment1=None, assessment2=None):
    lines = []
    lines.append('<table>')
    header = '<tr><th>#</th><th>Result URL</th>'
    if (assessment1):
        header += '<th>Assessment 1</th>'
    if (assessment2):
        header += '<th>Assessment 2</th>'
    header += '</tr>'
    lines.append(header)
    i = 0
    for url in urls:
        show_url = url
        if (len(url) > 80):
            show_url = url[:75] + '...'
        line = '<tr><td>{}</td><td><a href="{:s}">{:s}</a></td>'.format(i+1, url, show_url)
        if (assessment1):
            if (assessment1[i] == 0):
                line += '<td><em>Not relevant</em></td>'
            else:
                line += '<td><strong>Relevant</strong></td>'
        if (assessment2):
            if (assessment2[i] == 0):
                line += '<td><em>Not relevant</em></td>'
            else:
                line += '<td><strong>Relevant</strong></td>'
        line += '</tr>'
        lines.append(line)
        i = i+1
    lines.append('</table>')
    display( HTML(''.join(lines)) )

We can use this function to display a list of URLs, optionally together with one or two assessment lists:

In [15]:
print("Just a list of URLs:")
display_results(urls)

print("With one assessment:")
display_results(urls, my_assessment)

print("With two assessments:")
display_results(urls, my_assessment, another_assessment)

Just a list of URLs:


#,Result URL
1,https://en.wikipedia.org/wiki/Information_retrieval/
2,http://www.dictionary.com/browse/information
3,https://nlp.stanford.edu/IR-book/


With one assessment:


#,Result URL,Assessment 1
1,https://en.wikipedia.org/wiki/Information_retrieval/,Relevant
2,http://www.dictionary.com/browse/information,Not relevant
3,https://nlp.stanford.edu/IR-book/,Relevant


With two assessments:


#,Result URL,Assessment 1,Assessment 2
1,https://en.wikipedia.org/wiki/Information_retrieval/,Relevant,Not relevant
2,http://www.dictionary.com/browse/information,Not relevant,Not relevant
3,https://nlp.stanford.edu/IR-book/,Relevant,Relevant


Now we are ready to perform an actual evaluation, which will involve a substantial amount of manual work.

---

# Tasks

**Your name:** Enea Camishi

### Task 1

Think up and formulate a information need (for example in the field of Computer Science or Medicine) for which you think the answer can be found in scientific publications. On page 152 in the book an example of such an information need is shown: "Information on whether drinking red wine is more effective at reducing the risk of heart attacks than white wine."

**Answer:** What is the role of neural networks in developing bots for multiplayer games?

Next, write down specifically what documents have to look like to satisfy your information need. For example if your information need is about finding an overview of different cancer types, you could state that a document would need to list at least ten types of cancer to satisfy your information need (among other criteria). Write this down as a protocol with rules and examples. For example, such a protocol could state that at least three out of five given criteria have to be fulfilled for a document to be considered relevant for the information need, and then specify the criteria. Or your protocol could have the form of a sequence of rules, where each rule lets you either label the document as relevant or not relevant, or proceed with the next rule. Such rules and criteria can, for example, be about the general topic of the paper, the concepts mentioned in it, the covered relations between concepts, the type of publication (research paper, overview paper, etc.), the number of references, the types of contained diagrams, and so on, depending on your specified information need.

**Answer:** 

## Rules 

    Rule 1: Document should discuss the use of neural networks.
    Rule 2: Document should discuss multiplayer video games.
    Rule 3: The document must mention game bot behavior.
    Rule 4: The document should be a research publication, technical report, or review paper.
    Rule 5: The document must include examples or case studies of neural networks being applied to develop bots in games.


### Task 2

Formulate a keyword query that represents the information need. For the example on page 152 in the book (see above), the example query "wine AND red AND white AND heart AND attack AND effective" is given. (You don't need to use connectors like "AND", but if you do, make first sure your chosen search engines below actually support them.)

**Answer:** "neural networks multiplayer games bot behavior technical report"

Then submit your query to **two** of the following academic search engines:

- [Google Scholar](https://scholar.google.com) (all science disciplines)
- [Semantic Scholar](https://www.semanticscholar.org) (all science disciplines)
- [PubMed Search](https://www.ncbi.nlm.nih.gov/pubmed) (Life Sciences / biomedicine)

The right choice of two from the three search engine depends on the topic of your information need. If your information need is in the Life Sciences and biomedicine, it's probably best to include PubMed Search, but otherwise you should pick Google Scholar and Semantic Scholar.

Extract a list of the top 10 URLs of the lists of each of the search engines given the query. To be ensure that your results are reproducible, it is advised to use the private mode of your browser. Try to access the resulting publications. For the publications where that is not possible (because of dead links or because the publication is pay-walled even within the VU network), exclude them from the list and add more publications to the end of your list (that is, append results number 11, then 12, etc. to ensure you have two lists of 10 publications each). In order to deal with paywalls, you should try accessing the articles from the VU network, use
[UBVU Off-Campus
Access](http://www.ub.vu.nl.vu-nl.idm.oclc.org/nl/faciliteiten/toegang-buiten-de-campus/index.aspx), or try to find the respective documents from alternative sources (Google Scholar, for example, is very good at finding free PDFs of articles). If you get fewer than 10 results for one of the search engines, modify the keyword query above to make it more inclusive, and then redo the steps of this task.

Store your two lists of URLs in the form of Python lists as introduced above. Then, use the `display_results` function to nicely display them.

In [16]:
# Create two of the lists below, depending on your chosen engines:

urls_google = ["https://link.springer.com/article/10.1007/s11227-015-1545-2", 
               "https://www.inderscienceonline.com/doi/abs/10.1504/IJDMMM.2023.129963", 
               "https://dl.acm.org/doi/abs/10.1145/1653662.1653694",
               "https://dl.acm.org/doi/abs/10.1145/3219819.3219925",
               "https://ieeexplore.ieee.org/abstract/document/5679655",
               "https://dspace.bracu.ac.bd:8443/xmlui/handle/10361/11024",
               "https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=4fc701793a7efe4b59c58b071e302e0c1eb87616#page=182",
               "https://ieeexplore.ieee.org/abstract/document/8712055",
               "https://dl.acm.org/doi/abs/10.1145/3313831.3376223",
               "https://onlinelibrary.wiley.com/doi/full/10.1155/2022/9429475"]

urls_semantic = ["https://www.igi-global.com/gateway/article/76393",
                 "https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8924850",
                 "https://ieeexplore.ieee.org/document/8924850",
                 "https://ceur-ws.org/Vol-2350/short2.pdf",
                 "https://www.semanticscholar.org/reader/d5b38b1204b534d36c1782c30eacdb10ef440c25",
                 "https://link.springer.com/book/10.1007/3-540-45327-X",
                 "https://dl.acm.org/doi/10.1145/1146816.1146827",
                 "http://modelai.gettysburg.edu/2010/adaptiveAstar/AdaptiveAstar.pdf",
                 "https://www.researchgate.net/publication/314106580_Calm_down_buddy_it%27s_just_a_game_Behavioral_patterns_observed_among_teamwork_MMO_participants_in_WARGAMING%27s_world_of_tanks",
                 "https://www.semanticscholar.org/paper/Impact-of-Human-like-Cues-on-Human-Trust-in-Brain-Lee/ae7bce53e29d33a002f86e53f1617dc0173635d9"]

# Call display_results here
display_results(urls_google)
display_results(urls_semantic)

#,Result URL
1,https://link.springer.com/article/10.1007/s11227-015-1545-2
2,https://www.inderscienceonline.com/doi/abs/10.1504/IJDMMM.2023.129963
3,https://dl.acm.org/doi/abs/10.1145/1653662.1653694
4,https://dl.acm.org/doi/abs/10.1145/3219819.3219925
5,https://ieeexplore.ieee.org/abstract/document/5679655
6,https://dspace.bracu.ac.bd:8443/xmlui/handle/10361/11024
7,https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=4fc701793a7e...
8,https://ieeexplore.ieee.org/abstract/document/8712055
9,https://dl.acm.org/doi/abs/10.1145/3313831.3376223
10,https://onlinelibrary.wiley.com/doi/full/10.1155/2022/9429475


#,Result URL
1,https://www.igi-global.com/gateway/article/76393
2,https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8924850
3,https://ieeexplore.ieee.org/document/8924850
4,https://ceur-ws.org/Vol-2350/short2.pdf
5,https://www.semanticscholar.org/reader/d5b38b1204b534d36c1782c30eacdb10ef440c25
6,https://link.springer.com/book/10.1007/3-540-45327-X
7,https://dl.acm.org/doi/10.1145/1146816.1146827
8,http://modelai.gettysburg.edu/2010/adaptiveAstar/AdaptiveAstar.pdf
9,https://www.researchgate.net/publication/314106580_Calm_down_buddy_it%27s_j...
10,https://www.semanticscholar.org/paper/Impact-of-Human-like-Cues-on-Human-Tr...


### Task 3

Then, find a fellow student who will **independently**
assess the results as "relevant" or "not relevant" using the protocol that you
have defined above, and also help (at least) one other student for his/her
assessment. Write down their names here:

**Name of the student who assesses my results:** Kamil Pulchny

**Name of the student who I help to assess his/her results:** Kamil Pulchny

Show to the other assessor everything you have written down above for Tasks 1 and 2 (and you might also want to give him/her the PDFs you got for these papers to simplify the process).

You as assessors need to stick to the protocol you made in Task 1 and should not discuss with each other, especially when you doubt whether a result is relevant or not. Write down your assessments as lists of relevance values, as introduced above, and make sure they correctly map to the URLs by displaying them together with the `display_results` function.

To avoid problems with extreme results, mark in each list at least one paper as 'relevant' and at least one paper as 'not relevant'. That is, if all papers seem relevant, mark the one that seems least relevant 'not relevant', and conversely, if none of the papers seem relevant, mark the one that seems a bit more relevant than the others as 'relevant'.

In [17]:
# 0 = not relevant; 1 = relevant

# You only need to create 4 of the following 6 lists, again depending on which search engines you chose.

# Assessment 1 is from you:

assessment1_google = [1, 1, 1, 1, 1, 1, 0, 0, 1, 0]
assessment1_semantic = [0, 1, 1, 1, 0, 0, 1, 0, 0, 0]


# Assessment 2 is from your fellow student (don't show him/her your own assessment!):

assessment2_google = [1,0,1,1,0,0,0,1,1,0]
assessment2_semantic = [0,1,1,0,1,0,1,0,0,0]


# Call display_results here
display_results(urls_google, assessment1_google, assessment2_google)
display_results(urls_semantic, assessment1_semantic, assessment2_semantic)

#,Result URL,Assessment 1,Assessment 2
1,https://link.springer.com/article/10.1007/s11227-015-1545-2,Relevant,Relevant
2,https://www.inderscienceonline.com/doi/abs/10.1504/IJDMMM.2023.129963,Relevant,Not relevant
3,https://dl.acm.org/doi/abs/10.1145/1653662.1653694,Relevant,Relevant
4,https://dl.acm.org/doi/abs/10.1145/3219819.3219925,Relevant,Relevant
5,https://ieeexplore.ieee.org/abstract/document/5679655,Relevant,Not relevant
6,https://dspace.bracu.ac.bd:8443/xmlui/handle/10361/11024,Relevant,Not relevant
7,https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=4fc701793a7e...,Not relevant,Not relevant
8,https://ieeexplore.ieee.org/abstract/document/8712055,Not relevant,Relevant
9,https://dl.acm.org/doi/abs/10.1145/3313831.3376223,Relevant,Relevant
10,https://onlinelibrary.wiley.com/doi/full/10.1155/2022/9429475,Not relevant,Not relevant


#,Result URL,Assessment 1,Assessment 2
1,https://www.igi-global.com/gateway/article/76393,Not relevant,Not relevant
2,https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8924850,Relevant,Relevant
3,https://ieeexplore.ieee.org/document/8924850,Relevant,Relevant
4,https://ceur-ws.org/Vol-2350/short2.pdf,Relevant,Not relevant
5,https://www.semanticscholar.org/reader/d5b38b1204b534d36c1782c30eacdb10ef440c25,Not relevant,Relevant
6,https://link.springer.com/book/10.1007/3-540-45327-X,Not relevant,Not relevant
7,https://dl.acm.org/doi/10.1145/1146816.1146827,Relevant,Relevant
8,http://modelai.gettysburg.edu/2010/adaptiveAstar/AdaptiveAstar.pdf,Not relevant,Not relevant
9,https://www.researchgate.net/publication/314106580_Calm_down_buddy_it%27s_j...,Not relevant,Not relevant
10,https://www.semanticscholar.org/paper/Impact-of-Human-like-Cues-on-Human-Tr...,Not relevant,Not relevant


### Task 4

Compute Cohen's kappa to quantify how much the two assessors agreed. Use the function `cohen_kappa_score` demonstrated above to calculate two times the inter-annotator agreement (once for each of the two search engines), and print out the results.

In [18]:
# Add your code here:

kappa_google = cohen_kappa_score(assessment1_google, assessment2_google)
kappa_semantic = cohen_kappa_score(assessment1_semantic, assessment2_semantic)
#kappa_pubmed = ...

print("Kappa for Google Scholar:", kappa_google)
print("Kappa for Semantic Scholar:", kappa_semantic)
#print("Kappa for PubMed:", kappa_pubmed)

Kappa for Google Scholar: 0.19999999999999996
Kappa for Semantic Scholar: 0.5833333333333333


Explain whether the agreement can be considered high or not, based on the interpretation table on [this Wikipedia page](https://en.wikipedia.org/wiki/Fleiss'_kappa#Interpretation) (this Wikipedia page is about a different type of kappa but the interpretation table can also be used for Cohen's kappa).

**Answer:** The agreement cannot be considered high for google as it is between 0.01 and 0.2, counting as a poor agreement. Meanwhile semantic scholar COULD be considered moderate, but definitely not high.

### Task 5

Define a function called `precision_at_n` that calculates Precision@n as described in the lecture slides, which takes as input an assessment list and a value for _n_ and returns the respective Precision@n value. Run this function to calculate Precision@10 (that is, n=10) on all four assessments (two assessors and two search engines).

In [19]:
# Add your code here:

def precision_at_n(assessment_list, n):
    relevant_items = assessment_list[:n]
    precision = sum(relevant_items) / n
    return precision

# Print out Precision@10 for all assessments here.
print("Precision@10 for assessment1_google:", precision_at_n(assessment1_google, 10))
print("Precision@10 for assessment2_google:", precision_at_n(assessment2_google, 10))
print("Precision@10 for assessment1_semantic:", precision_at_n(assessment1_semantic, 10))
print("Precision@10 for assessment2_semantic:", precision_at_n(assessment2_semantic, 10))

Precision@10 for assessment1_google: 0.7
Precision@10 for assessment2_google: 0.5
Precision@10 for assessment1_semantic: 0.4
Precision@10 for assessment2_semantic: 0.4


Explain what these specific Precision@10 results tell us (or don't tell us) about the quality of the two search engines for your particular information need. You can also refer to the results of Task 4 if necessary.

**Answer:** Since precision@10 is higher at google scholar, it suggest that it returned more relevant documents than semantic sholar, however the difference in precision indicates a disagreement, which means there is inconsistencies in the relevance. For semantic scholar it is lower at 0.4 but they are more in agreement with each other.

# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done **individually**, and that code sharing or copying are **strictly forbidden** and will be punished.